In [2]:
import pandas as pd
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
df_dev  = pd.read_csv("../dados/ready/dev.csv")
df_comp = pd.read_csv("../dados/ready/comp.csv")

In [4]:
df_dev.columns

Index(['loan_id', 'amount', 'duration', 'payments', 'status',
       'acc_age_at_loan', 'is_account_shared', 'sex', 'age', 'num_inhab',
       'perc_urban_inhab', 'avg_salary', 'enterp_per_1000', 'num_crimes',
       'unemployment', 'unemployment_delta', 'crimes_delta',
       'type_card_classic', 'type_card_gold', 'type_card_junior',
       'type_card_other', 'has_card', 'min_balance', 'avg_balance',
       'max_balance', 'sanctions', 'only_to_na'],
      dtype='object')

In [5]:
df_comp.columns

Index(['loan_id', 'amount', 'duration', 'payments', 'status',
       'acc_age_at_loan', 'is_account_shared', 'sex', 'age', 'num_inhab',
       'perc_urban_inhab', 'avg_salary', 'enterp_per_1000', 'num_crimes',
       'unemployment', 'unemployment_delta', 'crimes_delta',
       'type_card_classic', 'type_card_gold', 'type_card_junior',
       'type_card_other', 'has_card', 'min_balance', 'avg_balance',
       'max_balance', 'sanctions', 'only_to_na'],
      dtype='object')

In [3]:
def get_x(df: pd.DataFrame) -> None:
    condition = (df.columns != 'status') & (df.columns != 'account_id') & (df.columns != 'loan_id') & (
        df.columns != 'client_id') & (df.columns != 'district_id') & (df.columns != 'disp_id') & (df.columns != 'district_id_y') & (
        df.columns != 'district_id_x') & (df.columns != 'id')
    return df.loc[:, condition]

def get_y(df: pd.DataFrame) -> None:
    return df.status

In [4]:
start = time.time()

train, test = train_test_split(df_dev, test_size = 0.2)

x = get_x(train)
y = get_y(train)

log_reg = KNeighborsClassifier()
log_reg.fit(x, y)

end = time.time()

pred_competition = log_reg.predict_proba(get_x(df_comp))

prediction = pd.DataFrame({'Id': df_comp['loan_id'], 'Predicted': pred_competition[::, 1]})

predicted = log_reg.predict_proba(get_x(test))[::, 1]
expected  = get_y(test)
score = roc_auc_score(expected, predicted)

print(f"Score is {score}")

print(f"Time Elapsed: {end-start}")

Score is 0.588392857142857
Time Elapsed: 0.005998373031616211


In [5]:
prediction.to_csv("../predictions/k-nearest.csv", index = False)